In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd S:\PPMI_RNA\Processed\Male
data = pd.read_csv('All_Counts_PatAndHealthy_Disease_Male.csv')
data['DIAGNOSIS'] = data['DIAGNOSIS'].replace({'PD': 1, 'Control': 0})
data.head(1)

S:\PPMI_RNA\Processed\Male


,HudAlphaID,DIAGNOSIS,PATNO,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000284332.1,ENSG00000237613.2,ENSG00000268020.3,...,ENSG00000273739.1,ENSG00000276700.1,ENSG00000276312.1,ENSG00000275757.1,ENSG00000278573.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278625.1,ENSG00000277374.1
0,5104_SL_0001,1,3174,15,232,15,0,0,0,0,...,0,1,16,1,0,2,19,0,0,0


In [3]:
X = data.drop(['HudAlphaID','PATNO'], axis = 1)
y = data['DIAGNOSIS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   DIAGNOSIS  ENSG00000223972.5  ENSG00000227232.5  ENSG00000278267.1  \
0          1                 15                232                 15   
1          1                  0                 97                  8   

   ENSG00000243485.5  ENSG00000284332.1  ENSG00000237613.2  ENSG00000268020.3  \
0                  0                  0                  0                  0   
1                  0                  0                  0                  0   

   ENSG00000240361.2  ENSG00000186092.6  ...  ENSG00000273739.1  \
0                  0                  0  ...                  0   
1                  0                  2  ...                  0   

   ENSG00000276700.1  ENSG00000276312.1  ENSG00000275757.1  ENSG00000278573.1  \
0                  1                 16                  1                  0   
1                  9                  0                  1                  0   

   ENSG00000276017.1  ENSG00000278817.1  ENSG00000277196.4  ENSG00000278625.1  \
0        

In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.725925925925926

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestClassifier(n_estimators=93,
                                          random_state=RandomState(MT19937) at 0x1FF26B83E40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1FF26B83E40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  32


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
0,DIAGNOSIS,1
47285,ENSG00000167434.9,1
35855,ENSG00000257878.1,1


### Top 50

In [10]:
Top50.to_csv ('DEGs_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('DEGs_Boruta_All_Male.csv', index = False)

In [11]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9824\1893064251.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000189337.16,ENSG00000076864.19,ENSG00000187010.20,ENSG00000222112.1,ENSG00000273136.6,ENSG00000281741.2,ENSG00000226067.6,ENSG00000263961.7,ENSG00000135778.11,...,ENSG00000258476.5,ENSG00000258831.1,ENSG00000262332.1,ENSG00000167434.9,ENSG00000280096.1,ENSG00000268240.1,ENSG00000268278.1,ENSG00000225178.5,ENSG00000100116.16,ENSG00000238222.3
0,1,359,1653,9,544,2592,88,776,55,1500,...,349,67,61,391,68,117,19,188,278,26
1,1,197,61,0,672,6702,22,489,96,1465,...,490,116,109,446,23,64,12,1137,76,9


### Top 100

In [12]:
Top100 = selected_rf_features.iloc[:100, :]
Top100.to_csv ('DEGs_Boruta_Top100_Male.csv', index = False)
Top100.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
47285,ENSG00000167434.9,1


In [13]:
# create a list of column names to subset from data
col_names = Top100.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top100_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9824\3442473894.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000189337.16,ENSG00000076864.19,ENSG00000187010.20,ENSG00000222112.1,ENSG00000116983.12,ENSG00000173212.4,ENSG00000273136.6,ENSG00000281741.2,ENSG00000226067.6,...,ENSG00000268433.1,ENSG00000225178.5,ENSG00000186431.18,ENSG00000105048.16,ENSG00000222365.1,ENSG00000100218.11,ENSG00000189060.5,ENSG00000100116.16,ENSG00000238222.3,ENSG00000275063.1
0,1,359,1653,9,544,392,104,2592,88,776,...,78,188,1520,181,251,48,1149,278,26,82
1,1,197,61,0,672,270,74,6702,22,489,...,36,1137,1603,88,242,46,441,76,9,2


### Top 250

In [14]:
Top250 = selected_rf_features.iloc[:250, :]
Top250.to_csv ('DEGs_Boruta_Top250_Male.csv', index = False)
Top250.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
47285,ENSG00000167434.9,1


In [15]:
# create a list of column names to subset from data
col_names = Top250.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top250_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9824\3939843383.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000189337.16,ENSG00000142794.18,ENSG00000076864.19,ENSG00000187010.20,ENSG00000159023.21,ENSG00000134686.18,ENSG00000222112.1,ENSG00000116983.12,ENSG00000199377.1,...,ENSG00000184792.15,ENSG00000189060.5,ENSG00000100116.16,ENSG00000102010.14,ENSG00000169891.17,ENSG00000241886.1,ENSG00000238222.3,ENSG00000157514.16,ENSG00000130830.14,ENSG00000275063.1
0,1,359,880,1653,9,147789,3026,544,392,25,...,4312,1149,278,275,5104,449,26,7394,25655,82
1,1,197,407,61,0,51779,3395,672,270,17,...,981,441,76,361,7006,827,9,6408,11059,2


### Top 500

In [16]:
Top500 = selected_rf_features.iloc[:250, :]
Top500.to_csv ('DEGs_Boruta_Top500_Male.csv', index = False)
Top500.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
47285,ENSG00000167434.9,1


In [17]:
# create a list of column names to subset from data
col_names = Top500.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top500_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9824\2027917441.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000189337.16,ENSG00000142794.18,ENSG00000076864.19,ENSG00000187010.20,ENSG00000159023.21,ENSG00000134686.18,ENSG00000222112.1,ENSG00000116983.12,ENSG00000199377.1,...,ENSG00000184792.15,ENSG00000189060.5,ENSG00000100116.16,ENSG00000102010.14,ENSG00000169891.17,ENSG00000241886.1,ENSG00000238222.3,ENSG00000157514.16,ENSG00000130830.14,ENSG00000275063.1
0,1,359,880,1653,9,147789,3026,544,392,25,...,4312,1149,278,275,5104,449,26,7394,25655,82
1,1,197,407,61,0,51779,3395,672,270,17,...,981,441,76,361,7006,827,9,6408,11059,2
